In [1]:
!pip install uv --quiet

In [ ]:
!uv pip install ngrok==1.4.0 fastapi==0.112.2 vllm==0.6.0 triton==3.0.0 --force-reinstall --system --quiet

In [ ]:
from kaggle_secrets import UserSecretsClient
import subprocess
import threading
import requests
import ngrok
import queue
import time
import os

In [ ]:
user_secrets = UserSecretsClient()

hf_token = user_secrets.get_secret("hf_token")
ngrok_token = user_secrets.get_secret("ngrok_token")

In [ ]:
ngrok.set_auth_token(ngrok_token)

In [ ]:
def start_ngrok(q):
    try:
        ngrok_listener = ngrok.forward("localhost:8000")
        q.put(ngrok_listener)
    except Exception as e:
        print(f"Error in start_ngrok: {e}")

In [ ]:
url_queue = queue.Queue()

ngrok_thread = threading.Thread(target=start_ngrok, args=(url_queue,))
ngrok_thread.start()

In [ ]:
while True:
    try:
        ngrok_listener = url_queue.get()
        public_url = ngrok_listener.url()
        if public_url:
            break
        print("Waiting for ngrok URL...")
        time.sleep(1)
    except Exception as e:
        print(f"Error in retrieving ngrok URL: {e}")

print("Ngrok tunnel established at:", public_url)

In [ ]:
chat_templates = ['phi-3.jinja', 'llama-3-instruct.jinja']

for model_ctemplate in chat_templates:
    response = requests.get(f'https://raw.githubusercontent.com/chujiezheng/chat_templates/main/chat_templates/{model_ctemplate}')

    with open(f'/kaggle/working/{model_ctemplate}', 'wb') as file:
        file.write(response.content)

In [ ]:
system_args = [
    "--download-dir", "/kaggle/working/models/",
    "--api-key", "open-source-model",
    "--tensor-parallel-size", "2"
]

In [ ]:
llama_31_8b_args = [
    "meta-llama/Meta-Llama-3.1-8B-Instruct",
    "--chat-template", "/kaggle/working/llama-3-instruct.jinja",
    "--max-model-len", "2048",
    "--dtype", "float16",
]

In [ ]:
phi35_mini_args = [
    "microsoft/Phi-3.5-mini-instruct",
    "--chat-template", "/kaggle/working/phi-3.jinja",
    "--max-model-len", "2048",
    "--dtype", "float16",
]

In [ ]:
vllm_env = os.environ.copy()
vllm_env['HF_TOKEN'] = hf_token

In [ ]:
# Starting vLLM serve
def vllm_serve(model_args):
    cmd = ["vllm", "serve"] + model_args + system_args
    
    with open('/kaggle/working/vllm.log', 'w') as log_file:
        process = subprocess.Popen(
            cmd,
            stdout=log_file,
            stderr=subprocess.STDOUT,
            text=True,
            bufsize=1,
            universal_newlines=True,
            env=vllm_env
        )
    
    return process

In [ ]:
process = vllm_serve(llama_31_8b_args)

In [ ]:
time.sleep(2*60)

In [ ]:
with open('/kaggle/working/vllm.log', 'r') as f:
    lines = f.readlines()
    for f in lines[-10:]:
        print(f, end="")

In [ ]:
# process.terminate()
# process.wait()